# Imports

In [1]:
# Uncomment and run the following commands if this is your first time running the code in this notebook:

# !pip install PyMuPDF 
# !pip install transformers torch
# !pip install --upgrade ipywidgets
# !python -m spacy download nl_core_news_sm

In [2]:
import fitz  # PyMuPDF
import torch
import logging
import spacy
import os
import string
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from sklearn.model_selection import train_test_split, LeaveOneOut, StratifiedKFold
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.pipeline import FeatureUnion
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, MaxAbsScaler
from scipy.sparse import hstack
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel, TFAutoModel
from collections import Counter

In [3]:
def extract_text_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    text = ""
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()
    return text

# Dataset Construction

In [4]:
directory = 'data'
df = pd.DataFrame(columns=['article_id' , 'paragraph_id', 'text', 'group', 'publication_date'])

print('Please ensure that only pdf files of articles are present in the subfolders of the specified directory')
article_nr = 1
for folder in os.listdir(directory):
    folder_size = len(os.listdir(directory + "\\" + folder))
    print(f'{folder_size} article(s) detected in {folder} folder')
    
    for article in os.listdir(directory + '\\' + folder):
        file_path = os.path.join(directory, folder, article)
        text = extract_text_from_pdf(directory + '\\' + folder + '\\' + article)
        date = article.split(' ')[-1].split('.')[0] #Remove the article number and ".pdf" to obtain the publication date
        
        paragraphs = [para.strip() for para in text.split("\n \n") if para.strip()]
        para_nr = 1
        for para in paragraphs:
            df_temp = pd.DataFrame([[article_nr, para_nr, para, folder, date, file_path]], 
                                   columns=['article_id' , 'paragraph_id', 'text', 'group', 'publication_date', 'file_path'])
            df = pd.concat([df, df_temp])
            para_nr += 1
        article_nr += 1
        
df.set_index(['article_id' , 'paragraph_id'], inplace=True)
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%d-%m-%Y')

Please ensure that only pdf files of articles are present in the subfolders of the specified directory
22 article(s) detected in Bouw & Vastgoed folder
33 article(s) detected in Handel & Industrie folder
34 article(s) detected in Zakelijke Dienstverlening folder
19 article(s) detected in Zorg folder


In [5]:
df

text  \
article_id paragraph_id                                                      
1          1             Provincies willen aan de slag met versoepeling...   
           2             Het draait allemaal om de drempelwaarde voor e...   
           3             Met een hogere drempelwaarde zouden minder ver...   
           4             In het hoofdlijnenakkoord hebben de vier coali...   
           5             De ondergrens is al langer onderwerp van discu...   
...                                                                    ...   
106        2             Telgenkamp vestigt haar hoop voor de korte ter...   
107        1             Waarom verzekeraars inkomsten uit zwart werk w...   
108        1             Verzekeraar wil klant helpen met zorgbemiddeli...   
           2             Verzekeraar wil wachtende patiënt aan snelle z...   
           3             Zorgbemiddeling is geen wondermiddel, maar kan...   

                                   group publication_date  \
article_id paragraph_id                                     
1          1             Bouw & Vastgoed       2024-08-28   
           2             Bouw & Vastgoed       2024-08-28   
           3             Bouw & Vastgoed       2024-08-28   
           4             Bouw & Vastgoed       2024-08-28   
           5             Bouw & Vastgoed       2024-08-28   
...                                  ...              ...   
106        2                        Zorg       2024-10-08   
107        1                        Zorg       2024-10-17   
108        1                        Zorg       2024-10-16   
           2                        Zorg       2024-10-16   
           3                        Zorg       2024-10-16   

                                                      file_path  
article_id paragraph_id                                          
1          1             data\Bouw & Vastgoed\1, 28-08-2024.pdf  
           2             data\Bouw & Vastgoed\1, 28-08-2024.pdf  
           3             data\Bouw & Vastgoed\1, 28-08-2024.pdf  
           4             data\Bouw & Vastgoed\1, 28-08-2024.pdf  
           5             data\Bouw & Vastgoed\1, 28-08-2024.pdf  
...                                                         ...  
106        2                        data\Zorg\7, 08-10-2024.pdf  
107        1                        data\Zorg\8, 17-10-2024.pdf  
108        1                        data\Zorg\9, 16-10-2024.pdf  
           2                        data\Zorg\9, 16-10-2024.pdf  
           3                        data\Zorg\9, 16-10-2024.pdf  

[332 rows x 4 columns]

In [6]:
for group in df['group'].unique():
    print(f"{len(df[df['group'] == group])} paragraphs in {group}.")

106 paragraphs in Bouw & Vastgoed.
101 paragraphs in Handel & Industrie.
71 paragraphs in Zakelijke Dienstverlening.
54 paragraphs in Zorg.


# Text pre-processing

Firstly, we load the nl_core_news_sm model and specify that [NEWLINE] should be treated as a single token.

In [7]:
# Load the 'nl_core_news_sm' model
nlp = spacy.load('nl_core_news_sm')

# Add [NEWLINE] as a single token so that it is not split into 3 seperate tokens
special_cases = {"[NEWLINE]": [{"ORTH": "[NEWLINE]"}]}
nlp.tokenizer.add_special_case("[NEWLINE]", [{"ORTH": "[NEWLINE]"}])

A pre-processed dataset df_clean is constructed out of df:

In [8]:
df_clean = df.copy()
df_clean['original_text'] = df_clean['text'].copy()

**Case Normalization**: <br>
- Lowercasing
- Replacing \n with '[NEWLINE]' 
- Removing duplicate spaces

In [9]:
def case_normalization(text):
    """Returns string of input containing only lowercase letters apart from [NEWLINE], which replaces \n"""
    text = text.lower()
    text = text.replace('\n', ' [NEWLINE] ')
    while text != text.replace('  ', ' '):
        text = text.replace('  ', ' ')
    return text

df_clean['text'] = df_clean['text'].apply(case_normalization)

**Punctuation Removal**

In [10]:
def remove_punctuation(text):
    """Returns the input text with all punctuation removed"""
    
    text = text.translate(text.maketrans("", "", string.punctuation))
    text = text.replace("NEWLINE", "[NEWLINE]")
    return text

df_clean['text'] = df_clean['text'].apply(remove_punctuation)

**Stop Word Removal**

Remove words that do not add semantic meaning to the text

In [11]:
# Sample text
text = "De snelle bruine vos springt over de luie hond."

# Process the text using spaCy
doc = nlp(text)

# Filter out stopwords
filtered_words = [token.text for token in doc if not token.is_stop]

# Join the filtered words back into a single string
text = " ".join(filtered_words)

# Print the result
print(text)

snelle bruine vos springt luie hond .


In [12]:
def remove_stopwords(text):
    """Returns string of input text with stopwords removed"""
    
    doc = nlp(text)
    filtered_words = [token.text for token in doc if not token.is_stop]
    text = " ".join(filtered_words)
    return text
    
    
# nlp = spacy.load("nl_core_news_sm")
df_clean['text'] = df_clean['text'].apply(remove_stopwords)

**Lemmatization**

In [13]:
# Sample text
text = "De katten liepen in de tuin."

# Process the text
doc = nlp(text)

# Print PoS tagging and Lemmatization for each token
print(f"{'Token':<15}{'PoS':<15}{'Lemma':<15}")
print("-" * 45)
for token in doc:
    print(f"{token.text:<15}{token.pos_:<15}{token.lemma_:<15}")

Token          PoS            Lemma          
---------------------------------------------
De             DET            de             
katten         NOUN           kat            
liepen         VERB           liepen         
in             ADP            in             
de             DET            de             
tuin           NOUN           tuin           
.              PUNCT          .              


In [14]:
doc = nlp(df_clean.loc[(1,1),'text'])

for token in doc:
    print(token.text, token.pos_, token.lemma_)
    print('')

provincies NOUN provincie

willen VERB willen

slag NOUN slag

versoepeling NOUN versoepeling

stikstofregels NOUN stikstofregel

[NEWLINE] SYM [NEWLINE]

kabinet NOUN kabinet

beoogde VERB beoogen

hogere ADJ hoog

drempelwaarde NOUN drempelwaarde

lijkt VERB lijken

rapport NOUN rapport

[NEWLINE] PRON [NEWLINE]

provincies NOUN provincie

willen VERB willen

‘ PUNCT ‘

voortvarend VERB voortvaren

’ NUM ’

slag NOUN slag

versoepeling NOUN versoepeling

stikstofregels NOUN stikstofregel

[NEWLINE] SYM [NEWLINE]

waarmee ADV waarmee

nieuwe ADJ nieuw

kabinet NOUN kabinet

nederland PROPN Nederland

slot NOUN slot

krijgen VERB krijgen

aannemelijk ADJ aannemelijk

[NEWLINE] SYM [NEWLINE]

belangrijke ADJ belangrijk

horde ADJ horde

stikstofcrisis ADJ stikstofcrisis

groot ADJ groot

aangenomen VERB aannemen

oordelen NOUN oordelen

[NEWLINE] SYM [NEWLINE]

wetenschappers NOUN wetenschapper

tno PRON tno

universiteit NOUN universiteit

amsterdam PROPN Amsterdam

onderzoek NOUN onde

In [15]:
def lemmatization(df, text_column="text", output_column="text"):
    """Lemmatizes the text in a specified column of a DataFrame and adds the results to a new column."""
    
    # Ensure the input column exists in the DataFrame
    if text_column not in df.columns:
        raise ValueError(f"Column '{text_column}' does not exist in the DataFrame.")
        
    # Apply SpaCy processing and lemmatization
    df[output_column] = df[text_column].apply(
        lambda text: " ".join([token.lemma_ for token in nlp(text) if not token.is_punct and not token.is_space]))
    
    return df

df_clean['text before lemmatization'] = df_clean['text'].copy()
df_clean = lemmatization(df_clean, text_column="text")
df_clean

text  \
article_id paragraph_id                                                      
1          1             provincie willen slag versoepeling stikstofreg...   
           2             draaien allemaal drempelwaran stikstofvergunni...   
           3             hoog drempelwaard vergunning [NEWLINE] aangevo...   
           4             hoofdlijnenakkoord vier coalitiepartij afsprek...   
           5             ondergren lang onderwerp discussie huidig Nede...   
...                                                                    ...   
106        2             telgenkamp vestigen hoop kort termijn twee cru...   
107        1             verzekeraar inkomst zwart werk vergoeden [NEWL...   
108        1             verzekeraar klant helpen zorgbemiddeling [NEWL...   
           2             verzekeraar wachten patiënt snel zorg helpen [...   
           3             zorgbemiddeling wondermiddel helpen zeggen Haa...   

                                   group publication_date  \
article_id paragraph_id                                     
1          1             Bouw & Vastgoed       2024-08-28   
           2             Bouw & Vastgoed       2024-08-28   
           3             Bouw & Vastgoed       2024-08-28   
           4             Bouw & Vastgoed       2024-08-28   
           5             Bouw & Vastgoed       2024-08-28   
...                                  ...              ...   
106        2                        Zorg       2024-10-08   
107        1                        Zorg       2024-10-17   
108        1                        Zorg       2024-10-16   
           2                        Zorg       2024-10-16   
           3                        Zorg       2024-10-16   

                                                      file_path  \
article_id paragraph_id                                           
1          1             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           2             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           3             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           4             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           5             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
...                                                         ...   
106        2                        data\Zorg\7, 08-10-2024.pdf   
107        1                        data\Zorg\8, 17-10-2024.pdf   
108        1                        data\Zorg\9, 16-10-2024.pdf   
           2                        data\Zorg\9, 16-10-2024.pdf   
           3                        data\Zorg\9, 16-10-2024.pdf   

                                                             original_text  \
article_id paragraph_id                                                      
1          1             Provincies willen aan de slag met versoepeling...   
           2             Het draait allemaal om de drempelwaarde voor e...   
           3             Met een hogere drempelwaarde zouden minder ver...   
           4             In het hoofdlijnenakkoord hebben de vier coali...   
           5             De ondergrens is al langer onderwerp van discu...   
...                                                                    ...   
106        2             Telgenkamp vestigt haar hoop voor de korte ter...   
107        1             Waarom verzekeraars inkomsten uit zwart werk w...   
108        1             Verzekeraar wil klant helpen met zorgbemiddeli...   
           2             Verzekeraar wil wachtende patiënt aan snelle z...   
           3             Zorgbemiddeling is geen wondermiddel, maar kan...   

                                                 text before lemmatization  
article_id paragraph_id                                                     
1          1             provincies willen slag versoepeling stikstofre...  
           2             draait allemaal drempelwaarde stikstofvergunni...  
           3             hogere drempelwaarde vergunningen [NEWLIN

**POS Tagging**. <br>
<br>
There are 2 types of POS tagging: <br>
- Rule-based POS tagging
- Statistical POS tagging 

**Benefits** of **rule-based** Part-of-speech (POS) tagging:
- Simple to implement and understand
- It doesn’t require a lot of computational resources or training data
- It can be easily customized to specific domains or languages

**Disadvantages** of **rule-based** Part-of-speech (POS) tagging:
- Less accurate than statistical taggers
- Limited by the quality and coverage of the rules
- It can be difficult to maintain and update

**Benefits** of **Statistical** Part-of-speech (POS) Tagging:
- More accurate than rule-based taggers
- Don’t require a lot of human-written rules
- Can learn from large amounts of training data

**Disadvantages** of **statistical** Part-of-speech (POS) Tagging:
- Requires more computational resources and training data
- It can be difficult to interpret and debug
- Can be sensitive to the quality and diversity of the training data

We select Statistical POS tagging since the accuracy tends to be higher and since pre-trained POS-models are avilable, the requirement for a lot of training data is no problem. Additionally, the required computational power is no problem due to the small size of the used data for this project. <br>
For more information on the used model, see https://github.com/evanmiltenburg/Dutch-tagger

**NOTE**: POS_tagging is also implicitly performed before lemmatization to improve lemmatization results 

In [16]:
def POS_tagging(text):
    """Returns a list of (token, POS tag) tuples for the input text"""
    doc = nlp(text)
    pos_tags = [(token.text, token.pos_) for token in doc]
    return pos_tags

df_clean['pos_tags'] = df_clean['text'].apply(POS_tagging)

**Resulting DataFrame**

In [17]:
df_clean

text  \
article_id paragraph_id                                                      
1          1             provincie willen slag versoepeling stikstofreg...   
           2             draaien allemaal drempelwaran stikstofvergunni...   
           3             hoog drempelwaard vergunning [NEWLINE] aangevo...   
           4             hoofdlijnenakkoord vier coalitiepartij afsprek...   
           5             ondergren lang onderwerp discussie huidig Nede...   
...                                                                    ...   
106        2             telgenkamp vestigen hoop kort termijn twee cru...   
107        1             verzekeraar inkomst zwart werk vergoeden [NEWL...   
108        1             verzekeraar klant helpen zorgbemiddeling [NEWL...   
           2             verzekeraar wachten patiënt snel zorg helpen [...   
           3             zorgbemiddeling wondermiddel helpen zeggen Haa...   

                                   group publication_date  \
article_id paragraph_id                                     
1          1             Bouw & Vastgoed       2024-08-28   
           2             Bouw & Vastgoed       2024-08-28   
           3             Bouw & Vastgoed       2024-08-28   
           4             Bouw & Vastgoed       2024-08-28   
           5             Bouw & Vastgoed       2024-08-28   
...                                  ...              ...   
106        2                        Zorg       2024-10-08   
107        1                        Zorg       2024-10-17   
108        1                        Zorg       2024-10-16   
           2                        Zorg       2024-10-16   
           3                        Zorg       2024-10-16   

                                                      file_path  \
article_id paragraph_id                                           
1          1             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           2             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           3             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           4             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           5             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
...                                                         ...   
106        2                        data\Zorg\7, 08-10-2024.pdf   
107        1                        data\Zorg\8, 17-10-2024.pdf   
108        1                        data\Zorg\9, 16-10-2024.pdf   
           2                        data\Zorg\9, 16-10-2024.pdf   
           3                        data\Zorg\9, 16-10-2024.pdf   

                                                             original_text  \
article_id paragraph_id                                                      
1          1             Provincies willen aan de slag met versoepeling...   
           2             Het draait allemaal om de drempelwaarde voor e...   
           3             Met een hogere drempelwaarde zouden minder ver...   
           4             In het hoofdlijnenakkoord hebben de vier coali...   
           5             De ondergrens is al langer onderwerp van discu...   
...                                                                    ...   
106        2             Telgenkamp vestigt haar hoop voor de korte ter...   
107        1             Waarom verzekeraars inkomsten uit zwart werk w...   
108        1             Verzekeraar wil klant helpen met zorgbemiddeli...   
           2             Verzekeraar wil wachtende patiënt aan snelle z...   
           3             Zorgbemiddeling is geen wondermiddel, maar kan...   

                                                 text before lemmatization  \
article_id paragraph_id                                                      
1          1             provincies willen slag versoepeling stikstofre...   
           2             draait allemaal drempelwaarde stikstofvergunni...   
           3             hogere drempelwaarde vergunningen [NE

# Saving Data

In [18]:
df_clean.to_csv('data\\pre-processed data.csv', index=True)  

# Testing Alternative Method for POS and lemmatization

**DID NOT IMPROVE PERFORMANCE!**<br>
The code below is only there for completeness sake, it did not improve performance.

We test if performance increases when POS tags are explicitely used to reinforce lemmatization

**NOTE:** POS-tagging occurs twice in the pre-processing: Once before lemmatization and once after. The first POS-tagging results are used to reinforce the lemmatization by providing more detailed input. After Lemmatization, POS-tagging are once again obtained to ensure that the final POS-tags match the final text.

In [19]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import nltk

# Uncomment and execute the 2 lines below to install the required nltk files, which only needs to be done once.
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [20]:
df_clean_experimental = df_clean.copy()

In [21]:
# Initialize NLTK lemmatizer
lemmatizer = WordNetLemmatizer()

In [22]:
# Function to convert spaCy POS to WordNet POS (needed for accurate lemmatization)
def spacy_to_wordnet_pos(spacy_pos):
    if spacy_pos.startswith('N'):  # Noun
        return wordnet.NOUN
    elif spacy_pos.startswith('V'):  # Verb
        return wordnet.VERB
    elif spacy_pos.startswith('J'):  # Adjective
        return wordnet.ADJ
    elif spacy_pos.startswith('R'):  # Adverb
        return wordnet.ADV
    else:
        return wordnet.NOUN  # Default to noun

In [23]:
def lemmatize_with_pos(pos_tags):
    return " ".join([lemmatizer.lemmatize(word, spacy_to_wordnet_pos(pos)) for word, pos in pos_tags])
    
def get_pos_tags_after_lemmatization(lemmatized_text):
    doc = nlp(lemmatized_text)
    return [(token.text, token.pos_) for token in doc]

In [24]:
df_clean_experimental['pos_tags_before_lemmatization'] = df_clean_experimental['text before lemmatization'].apply(POS_tagging)
print(df_clean_experimental['pos_tags_before_lemmatization'])

article_id  paragraph_id
1           1               [(provincies, NOUN), (willen, VERB), (slag, NO...
            2               [(draait, VERB), (allemaal, ADV), (drempelwaar...
            3               [(hogere, ADJ), (drempelwaarde, VERB), (vergun...
            4               [(hoofdlijnenakkoord, INTJ), (vier, NUM), (coa...
            5               [(ondergrens, NOUN), (langer, ADJ), (onderwerp...
                                                  ...                        
106         2               [(telgenkamp, NOUN), (vestigt, VERB), (hoop, N...
107         1               [(verzekeraars, NOUN), (inkomsten, NOUN), (zwa...
108         1               [(verzekeraar, ADJ), (klant, NOUN), (helpen, V...
            2               [(verzekeraar, ADJ), (wachtende, VERB), (patië...
            3               [(zorgbemiddeling, NOUN), (wondermiddel, NOUN)...
Name: pos_tags_before_lemmatization, Length: 332, dtype: object


In [25]:
df_clean_experimental['text'] = df_clean_experimental['pos_tags_before_lemmatization'].apply(lemmatize_with_pos)
print(df_clean_experimental['text'])

article_id  paragraph_id
1           1               provincies willen slag versoepeling stikstofre...
            2               draait allemaal drempelwaarde stikstofvergunni...
            3               hogere drempelwaarde vergunningen [NEWLINE] aa...
            4               hoofdlijnenakkoord vier coalitiepartijen afges...
            5               ondergrens langer onderwerp discussie huidige ...
                                                  ...                        
106         2               telgenkamp vestigt hoop korte termijn twee cru...
107         1               verzekeraars inkomsten zwart werk vergoeden [N...
108         1               verzekeraar klant helpen zorgbemiddeling [NEWL...
            2               verzekeraar wachtende patiënt snelle zorg help...
            3               zorgbemiddeling wondermiddel helpen zegt haarl...
Name: text, Length: 332, dtype: object


In [26]:
df_clean_experimental['pos_tags_after_lemmatization'] = df_clean_experimental['text'].apply(get_pos_tags_after_lemmatization)
print(df_clean_experimental['pos_tags_after_lemmatization'])

article_id  paragraph_id
1           1               [(provincies, NOUN), (willen, VERB), (slag, NO...
            2               [(draait, VERB), (allemaal, ADV), (drempelwaar...
            3               [(hogere, ADJ), (drempelwaarde, VERB), (vergun...
            4               [(hoofdlijnenakkoord, INTJ), (vier, NUM), (coa...
            5               [(ondergrens, NOUN), (langer, ADJ), (onderwerp...
                                                  ...                        
106         2               [(telgenkamp, NOUN), (vestigt, VERB), (hoop, N...
107         1               [(verzekeraars, NOUN), (inkomsten, NOUN), (zwa...
108         1               [(verzekeraar, ADJ), (klant, NOUN), (helpen, V...
            2               [(verzekeraar, ADJ), (wachtende, VERB), (patië...
            3               [(zorgbemiddeling, NOUN), (wondermiddel, NOUN)...
Name: pos_tags_after_lemmatization, Length: 332, dtype: object


In [27]:
df_clean_experimental

text  \
article_id paragraph_id                                                      
1          1             provincies willen slag versoepeling stikstofre...   
           2             draait allemaal drempelwaarde stikstofvergunni...   
           3             hogere drempelwaarde vergunningen [NEWLINE] aa...   
           4             hoofdlijnenakkoord vier coalitiepartijen afges...   
           5             ondergrens langer onderwerp discussie huidige ...   
...                                                                    ...   
106        2             telgenkamp vestigt hoop korte termijn twee cru...   
107        1             verzekeraars inkomsten zwart werk vergoeden [N...   
108        1             verzekeraar klant helpen zorgbemiddeling [NEWL...   
           2             verzekeraar wachtende patiënt snelle zorg help...   
           3             zorgbemiddeling wondermiddel helpen zegt haarl...   

                                   group publication_date  \
article_id paragraph_id                                     
1          1             Bouw & Vastgoed       2024-08-28   
           2             Bouw & Vastgoed       2024-08-28   
           3             Bouw & Vastgoed       2024-08-28   
           4             Bouw & Vastgoed       2024-08-28   
           5             Bouw & Vastgoed       2024-08-28   
...                                  ...              ...   
106        2                        Zorg       2024-10-08   
107        1                        Zorg       2024-10-17   
108        1                        Zorg       2024-10-16   
           2                        Zorg       2024-10-16   
           3                        Zorg       2024-10-16   

                                                      file_path  \
article_id paragraph_id                                           
1          1             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           2             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           3             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           4             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
           5             data\Bouw & Vastgoed\1, 28-08-2024.pdf   
...                                                         ...   
106        2                        data\Zorg\7, 08-10-2024.pdf   
107        1                        data\Zorg\8, 17-10-2024.pdf   
108        1                        data\Zorg\9, 16-10-2024.pdf   
           2                        data\Zorg\9, 16-10-2024.pdf   
           3                        data\Zorg\9, 16-10-2024.pdf   

                                                             original_text  \
article_id paragraph_id                                                      
1          1             Provincies willen aan de slag met versoepeling...   
           2             Het draait allemaal om de drempelwaarde voor e...   
           3             Met een hogere drempelwaarde zouden minder ver...   
           4             In het hoofdlijnenakkoord hebben de vier coali...   
           5             De ondergrens is al langer onderwerp van discu...   
...                                                                    ...   
106        2             Telgenkamp vestigt haar hoop voor de korte ter...   
107        1             Waarom verzekeraars inkomsten uit zwart werk w...   
108        1             Verzekeraar wil klant helpen met zorgbemiddeli...   
           2             Verzekeraar wil wachtende patiënt aan snelle z...   
           3             Zorgbemiddeling is geen wondermiddel, maar kan...   

                                                 text before lemmatization  \
article_id paragraph_id                                                      
1          1             provincies willen slag versoepeling stikstofre...   
           2             draait allemaal drempelwaarde stikstofvergunni...   
           3             hogere drempelwaarde vergunningen [NE